In [1]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'g:\\Il mio Drive\\PROJECTS\\Kidney-Tumor-Detection-End-To-End'

Same thing -> follow workflow on readme

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    # Parameters from params.yaml
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

#### update configuration manager

In [6]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Kidney Images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

#### update components

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    

    # load model from artifacts folder 
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )



    # function for creating data splits using keras in-built
    # copied and pasted from keras documentation (keras -> train valid generator)
    def train_valid_generator(self):
        
        # augmentation and rescaling operations
        datagenerator_kwargs = dict(
            rescale = 1./255,           # rescaling images to have pixel values in range [0,1]
            validation_split=0.20)      # validation test size

        # params used for data generators for both training and validation
        dataflow_kwargs = dict(
            # set the specified target size for the images = all dimensions except the last one
            # common practice for specifying the height and width of images while leaving the 
            # number of channels (RGB channels) unchanged
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear")
            # image resizing parameter -> bilinear interpolation takes a weighted average 
            # of four pixels around the target pixel to resize the image

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    

    # train and then save the model
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    # training function
    def train(self):
        # steps for image classification:
        # number of steps (or batches) the model should take in each epoch during the training phase
        # ensuring Efficient Dataset Processing (batches) and Complete Dataset Coverage in one epoch
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        # number of steps for the validation phase -> Monitoring Model Performance + Preventing Overfitting
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2023-11-17 21:24:28,715: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-17 21:24:28,741: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-17 21:24:28,747: INFO: common: created directory at: artifacts]
[2023-11-17 21:24:28,757: INFO: common: created directory at: artifacts\training]
Found 1471 images belonging to 2 classes.
Found 5889 images belonging to 2 classes.
368/368 [==============================] - 861s 2s/step - loss: 10.5595 - accuracy: 0.6196 - val_loss: 5.1304 - val_accuracy: 0.7170


Only training for 1 epoch because otherwise my CPU will literally cry.